# Milestone Project 2 - Blackjack Game
In this milestone project you will be creating a Complete BlackJack Card Game in Python.

Here are the requirements:

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

And most importantly:

* **You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!**


Feel free to expand this game. Try including multiple players. Try adding in Double-Down and card splits! Remember to you are free to use any resources you want and as always:

# HAVE FUN!

In [2]:
from random import shuffle
import numpy as np
from IPython.display import clear_output

In [19]:
# Single card class
class Card():
    
    def __init__(self, suit, val):
        self.suit = suit
        self.val = val
    
    def show(self):
        print(f"{self.val} of {self.suit}")
    
# Deck of 52 cards   
class Deck():
    
    def __init__(self):
        self.cards = []
        self.build()
    
    def __len__(self):
        return len(self.cards)
        
    def build(self):
        suits = ['clubs', 'diamonds', 'hearts', 'spades']
        values = list(range(2,11)) + ['jack', 'queen', 'king', 'ace']
        for suit in suits:
            for val in values:
                self.cards.append(Card(suit, val))
    
    def shuffle(self):
        shuffle(self.cards)
        
    def draw_card(self):
        if len(self.cards) > 0:
            return self.cards.pop()
        else:
            return 'No more cards in the deck.'

# Player class
class Player():
    
    cards_conv = {'jack':10, 'queen':10, 'king':10, 'ace':(1,11)}
    
    def __init__(self, name):
        self.name = name
        self.cards = [] # list of tuples [('val', 'suit'), ('val', 'suit')]
        
    def check_ace(self):
        while True:
            try:
                value = int(input(f"{self.name} has an ace, type 1 or 11: "))
                assert value == 1 or value == 11
            except:
                print("Please, type 1 or 11.")
            else:
                return value
                
    def sum_cards(self):
        score = 0
        for card in self.cards:
            card_val = card[0]
            if type(card_val) == str:
                card_val = self.cards_conv[card_val]
                if type(card_val) == tuple:
                    card_val = self.check_ace()
            score += card_val
        return score

class Human(Player):
    
    def __init__(self, name, budget, bet):
        Player.__init__(self, name)
        self.budget = budget
        self.bet = bet
        if self.bet > self.budget:
            print(f"You don't have enough money. Budget available: {self.budget}.")
        else:
            self.budget -= self.bet
            print(f"You bet is {self.bet}$. Your budget is {self.budget}$.")
        
    def apply_bet(self, new_bet):
        if new_bet > self.budget:
            print(f"You don't have enough money. Budget available: {self.budget}.")
            return False
        else:
            self.bet = new_bet
            self.budget -= self.bet
            print(f"You bet is {self.bet}$. Your budget is {self.budget}$.")
            return True
        
        print(f"New player {self.name} starts the game with a bet {self.bet}$.")
    

In [20]:
dealer = Player('Dealer')
player = Human(name='Peter', budget=100, bet=50)

You bet is 50$. Your budget is 50$.


In [21]:
def replay():
    while True:
        answer = input("Do you want to replay ('y' or 'n')? ")
        if answer.lower() == 'y' or answer.lower() == 'n':
            return answer.lower() == 'y'
        
# GAME's SETUP
game_on = True

while game_on:
    
    deck = Deck()
    deck.shuffle()
    
    dealer.cards = []
    player.cards = []
    if player.bet == 0:
        while True:
            desired_bet = int(input("How much do you want to bet? "))
            if player.apply_bet(desired_bet):
                break

    for i in range(2):
        # dealer draws two cards
        drawn_card = deck.draw_card()
        dealer.cards.append( (drawn_card.val, drawn_card.suit) )

        # player draws two cards
        drawn_card = deck.draw_card()
        player.cards.append( (drawn_card.val, drawn_card.suit) )

    print("The game begins!")
    print(f"\nDealer's cards: {dealer.cards[0]}, ('value', 'suit')")
    print(f"Player's cards: {player.cards}")
    print(f"Your current some is: {player.sum_cards()}\n")
    print(f"Number of cards in the deck: {len(deck)}")
    print('---------------------------------------------')


    
    check = True
    turn = 'player'
    
    # Player plays
    while turn == 'player':
        player_sum = player.sum_cards()
        ask = True
        while ask:
            try:
                draw_decision =  input("Do you want to draw a new card (type 'y' or 'n')? ")
                assert draw_decision.lower() == 'y' or draw_decision.lower() == 'n'
            except:
                print("Please, type 'y' or 'n'.")
            else:
                ask = False
                
        if draw_decision == 'y':
            drawn_card = deck.draw_card()
            player.cards.append( (drawn_card.val, drawn_card.suit) )
            player_sum = player.sum_cards()
            print(f"Player's cards: {player.cards}\n")
            print(f"Your current some is: {player_sum}\n")
            print(f"Number of cards in the deck: {len(deck)}")
            print('---------------------------------------------')
            
            ##### I. Player busts (exceeds 21) #####
            if player_sum > 21:
                ask = False
                game_on = False
                break
            ask = True
        
        else:
            print(f"Player's cards: {player.cards}\n")
            print(f"Your current some is: {player_sum}\n")
            print(f"Number of cards in the deck: {len(deck)}")
            print('---------------------------------------------')
            print("Passing to a dealer's turn...\n")
            turn = 'dealer'
        
    
    # Dealer plays
    while turn == 'dealer':
        dealer_sum = dealer.sum_cards()
        if dealer_sum == 21:
            game_on = False
            break
        
        ask = True
        while ask:
            if dealer_sum in range(18, 21):
                draw_decision = int(np.random.choice([0,1], 1, p=[0.9, 0.1]))
            elif dealer_sum in range(15, 18):
                draw_decision = int(np.random.choice([0,1], 1, p=[0.5, 0.5]))
            elif dealer_sum in range(12, 15):
                draw_decision = int(np.random.choice([0,1], 1, p=[0.1, 0.9]))
            else:
                draw_decision = 1
            ask = False
        
        if draw_decision:
            drawn_card = deck.draw_card()
            dealer.cards.append( (drawn_card.val, drawn_card.suit) )
            dealer_sum = dealer.sum_cards()
            print(f"Dealer draws.\nDealer's cards: {dealer.cards}\n")
            print(f"His current some is: {dealer_sum}\n")
            print(f"Number of cards in the deck: {len(deck)}")
            print('---------------------------------------------')
            
            ##### II. Dealer busts #####
            if dealer_sum > 21:
                ask = False
                game_on = False
                break
            ask = True
        
        else:
            print("Dealer finishes his turn.\n")
            print(f"Dealer's cards: {dealer.cards}\n")
            print(f"His current some is: {dealer_sum}\n")
            print(f"Number of cards in the deck: {len(deck)}")
            print('---------------------------------------------')
            print("Let's check the result...\n")
            game_on = False
            break


    ##### III. Nobody busts, the winner is the one who is closer to 21 #####
    if check:
        if player_sum > 21:
            print(f"Your sum of cards have exceeded 21. Bust! Dealer won. You lost your bet.")
            player.bet = 0
            
        elif dealer_sum > 21:
            print(f"His sum of cards have exceeded 21. Bust! Player won. He doubles his bet.")
            player.budget += player.bet * 2
            player.bet = 0
        
        elif player_sum == dealer_sum:
            print(f"Nobody won. Dealer's sum {dealer_sum}. Player's sum {player_sum}.")
            player.budget += player.bet
            player.bet = 0
            
        elif (21-dealer_sum) < (21-player_sum):
            print(f"Dealer won. You lost your bet.\nDealer's sum {dealer_sum} is closer to 21 than player's sum {player_sum}.")
            player.bet = 0
            
        else:
            print(f"Player won. You doubled your bet.\nPlayer's sum {player_sum} is closer to 21 than dealer's sum {dealer_sum}.")
            player.budget += player.bet * 2
            player.bet = 0 

    game_on = replay()
    if game_on:
        clear_output()
    else:
        print("Thank you for the game! Bye!")
        
    

How much do you want to bet? 899
You don't have enough money. Budget available: 250.
How much do you want to bet? 50
You bet is 50$. Your budget is 200$.
The game begins!

Dealer's cards: ('queen', 'spades'), ('value', 'suit')
Player's cards: [('queen', 'hearts'), ('queen', 'clubs')]
Your current some is: 20

Number of cards in the deck: 48
---------------------------------------------
Do you want to draw a new card (type 'y' or 'n')? n
Player's cards: [('queen', 'hearts'), ('queen', 'clubs')]

Your current some is: 20

Number of cards in the deck: 48
---------------------------------------------
Passing to a dealer's turn...

Dealer finishes his turn.

Dealer's cards: [('queen', 'spades'), (8, 'diamonds')]

His current some is: 18

Number of cards in the deck: 48
---------------------------------------------
Let's check the result...

Player won. You doubled your bet.
Player's sum 20 is closer to 21 than dealer's sum 18.
Do you want to replay ('y' or 'n')? n
Thank you for the game! By

In [22]:
player.budget

300